In [1]:
import torch 
import json,pickle,math
import pandas as pd
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
LSM = pickle.load(open('./kiba_ligand_similarity_matrix.pkl', 'rb'))
PSM = pickle.load(open('./kiba_protein_similarity_matrix.pkl', 'rb'))
full_df = pd.read_csv(open('./kiba_all_pairs.csv','r'))

In [3]:
SMILES = json.load(open('./data/KIBA/SMILES.txt'))
TARGETS = json.load(open('./data/KIBA/target_seq.txt'))
SMILES=list(SMILES.values())
TARGETS=list(TARGETS.values())

In [4]:
all_3_folds={}
for i in [0,1,2]:
    file_name = 'fold' +str(i) 

    temp = open('./data/kiba/KIBA_3_FOLDS/' + file_name +'.pkl', 'rb')
    new_df = pd.read_pickle(temp)
    all_3_folds.update({file_name:new_df})
    temp.close()

In [5]:
# all_3_folds['fold2']

In [6]:
def create_kiba_test_train(fold_number, all_3_folds):
    test_set = pd.DataFrame(columns = full_df.columns)
    train_set = pd.DataFrame(columns= full_df.columns)
    for i in [0,1,2]:
        fold_name = 'fold' + str(i) 
        df = all_3_folds[fold_name]
        if str(i) == fold_number:
            test_set =df.copy()
        if str(i) != fold_number:
            train_set = pd.concat([train_set, df.copy()], ignore_index=True)
    return train_set, test_set

# Create train test split on these 3 folds
## fold_number is the id of fold. For example, test = fold0, train = fold 1,2

In [7]:
fold_number='0'

In [8]:
train, test = create_kiba_test_train(fold_number=fold_number, all_3_folds=all_3_folds)

In [9]:
len(train) + len(test)

118254

In [10]:
# test_smiles = list(test['SMILES'])
# test_targets = list(test['Target Sequence'])
# train_smiles = list(train['SMILES'])
# train_targets = list(train['Target Sequence'])

# # for i in test_smiles:
# #     if i in train_smiles:
# #         print("common entity present")
# for i in test_targets:
#     if i in train_targets:
#         print("common entity present")


In [10]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 20
# num_classes = 10
batch_size = 12
learning_rate = 0.001

In [11]:
class custom_dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, smiles, targets, LSM,PSM,transform=None):
        self.df = dataframe
#         self.root_dir = root_dir
        self.smiles =smiles
        self.targets = targets
        self.LSM = LSM
        self.PSM = PSM
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        smi = self.df.iloc[idx]['SMILES']
        seq = self.df.iloc[idx]['Target Sequence']
        s_i = self.smiles.index(smi)
        t_i = self.targets.index(seq)
        
        ki=self.LSM[s_i]
        kj=self.PSM[t_i]
        
        ki_x_kj = np.outer(ki,kj)
        ki_x_kj = torch.tensor([ki_x_kj])
        output = {'outer_product': ki_x_kj , 'Label':self.df.iloc[idx]['Label']}
        return output

In [12]:
test_dataset = custom_dataset(dataframe=test, smiles=SMILES, targets = TARGETS, LSM=LSM,PSM=PSM)
train_dataset = custom_dataset(dataframe=train, smiles=SMILES, targets = TARGETS, LSM=LSM,PSM=PSM)


In [13]:
train_loader= torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader= torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [14]:
print(len(train_loader)*12, len(test_loader)*12)

68124 50148


In [15]:
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1,32, 5).double()
        self.pool1 = nn.MaxPool2d(2,2).double()
        self.conv2 = nn.Conv2d(32,18,3).double()
        self.pool2 = nn.MaxPool2d(2,2).double()
        self.fc1 = nn.Linear(18*525*55, 128).double()
        self.fc2 = nn.Linear(128,1).double()
        self.dropout = nn.Dropout(0.1).double()
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,18*525*55)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        
        return x
    

In [16]:
# for i in test_loader:
#     a = i['outer_product']
#     b= i['Label']
#     break
# # print(a)
# conv1 = nn.Conv2d(1,32,5).double()
# pool = nn.MaxPool2d(2,2).double()
# conv2 = nn.Conv2d(32,18,3).double()
# fc1 = nn.Linear(18*525*55, 128).double()
# fc2 = nn.Linear(128,1).double()
# dropout = nn.Dropout(0.1).double()
# x= conv1(a)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x= conv2(x)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x = x.view(-1,18*525*55)
# print(x.shape)
# x = dropout(x)
# print(x.shape)
# x = fc1(x)
# print(x.shape)
# x = fc2(x)
# print(x.shape)

In [20]:
model = ConvNet().to(device)

In [21]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Evaluation metrics


In [22]:
def rmse(y,f):
    rmse = math.sqrt(((y - f)**2).mean(axis=0))
    return rmse
def mse(y,f):
    mse = ((y - f)**2).mean(axis=0)
    return mse
def pearson(y,f):
    rp = np.corrcoef(y, f)[0,1]
    return rp
from lifelines.utils import concordance_index
def ci(y,f):
    return concordance_index(y,f)

# Train the model


In [23]:
def predicting(model, device, test_loader):
    model.eval()
    total_preds = np.array([])
    total_labels = np.array([])
    with torch.no_grad():
        correct = 0
        total = 0
        for i in test_loader:
            images = i['outer_product']
            labels = i['Label']
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images) 
            outputs = outputs.cpu().detach().numpy().flatten()
            labels =labels.cpu().detach().numpy().flatten()
            P = np.concatenate([total_preds, outputs])
            G = np.concatenate([total_labels, labels])
        
    return G, P

In [ ]:
# Train the model
best_mse = 1000
best_ci = 0
model_file_name = 'best_sim-CNN-DTA_kiba.model'
result_file_name = 'best_result_sim-CNNDTA_kiba.csv'
total_step = len(train_loader)
for epoch in range(num_epochs):
    c=0
    for i in train_loader:
        c=c+1
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.flatten(), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, c, total_step, loss.item()))
    
    # taking best model so far
#     G,P = predicting(model, device, test_loader)
#     ret = [rmse(G, P), mse(G, P), pearson(G, P), ci(G, P)]
#     if ret[1] < best_mse:
#         torch.save(model.state_dict(), model_file_name)
#         with open(result_file_name, 'w') as f:
#             f.write(','.join(map(str, ret)))
#         best_epoch = epoch+1
#         best_mse = ret[1]
#         best_ci = ret[-1]
#         best_r = ret[2]
        
#         print('rmse improved at epoch ', best_epoch,
#                       '; best_mse,best_ci,best_r:', best_mse, best_ci,best_r)
        
        

In [24]:
model.load_state_dict(torch.load('./SAVED_MODELS/final_sim-CNN-DTA_kiba0.model'))

<All keys matched successfully>

In [ ]:
model.eval()
# eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
total_preds = np.array([])
total_labels = np.array([])
with torch.no_grad():
    correct = 0
    total = 0
    c=0
    for i in test_loader:
        print(c)
        c=c+1
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images) 
        outputs = outputs.cpu().detach().numpy().flatten()
        labels =labels.cpu().detach().numpy().flatten()
        total_preds = np.concatenate([total_preds, outputs])
        total_labels = np.concatenate([total_labels, labels])
#         total_preds = torch.cat(total_preds, outputs.cpu(), 0 )
#         total_labels = torch.cat(total_labels, labels.cpu(), 0)
#         break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
G,P = total_labels, total_preds

In [ ]:
rmse(G,P)

In [ ]:
mse(G,P)

In [ ]:
pearson(G,P)

In [ ]:
ci(G,P)